# ResNet

upvote if find anything helpful

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import tensorflow.keras
from tensorflow.keras.layers import Conv2D, Dropout, MaxPool2D, AvgPool2D, Add, Dense 
from tensorflow.keras.preprocessing.image import load_img, img_to_array

import warnings
warnings.warn("ignore")
import os
os.chdir('../')
os.listdir('input')

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: ignore
  # This is added back by InteractiveShellApp.init_path()


['intel-image-classification']

# 1) convert image to array

## 1.1) train set

In [2]:
from keras.preprocessing.image import load_img, img_to_array
import os

def convert(path,y):
    array=[]
    img_cat = []
    for img_path in os.listdir(path):
        img = load_img(path + img_path, target_size=(150,150))
        img = img_to_array(img)
        img = img/255.
        array.append(img)
        img_cat.append(y)
    return np.array(array), np.array(img_cat)

Using TensorFlow backend.


In [3]:
trainX_building, trainY_building  = convert("input/intel-image-classification/seg_train/seg_train/buildings/",0)
trainX_forest,trainY_forest  = convert("input/intel-image-classification/seg_train/seg_train/forest/",1)
trainX_glacier,trainY_glacier  = convert("input/intel-image-classification/seg_train/seg_train/glacier/",2)
trainX_mount,trainY_mount  = convert("input/intel-image-classification/seg_train/seg_train/mountain/",3)
trainX_sea,trainY_sea  = convert("input/intel-image-classification/seg_train/seg_train/sea/",4)
trainX_street,trainY_street  = convert("input/intel-image-classification/seg_train/seg_train/street/",5)

print('train building shape ', trainX_building.shape, trainY_building.shape) 
print('train forest', trainX_forest.shape ,trainY_forest.shape)
print('train glacier', trainX_glacier.shape,trainY_glacier.shape)
print('train mountain', trainX_mount.shape, trainY_mount.shape)
print('train sea',     trainX_sea.shape, trainY_sea.shape)
print('train street', trainX_street.shape ,trainY_street.shape)

train building shape  (2191, 150, 150, 3) (2191,)
train forest (2271, 150, 150, 3) (2271,)
train glacier (2404, 150, 150, 3) (2404,)
train mountain (2512, 150, 150, 3) (2512,)
train sea (2274, 150, 150, 3) (2274,)
train street (2382, 150, 150, 3) (2382,)


In [4]:
X_train= np.concatenate((trainX_building,trainX_forest, trainX_glacier,trainX_mount, trainX_sea,trainX_street),axis=0)
y_train= np.concatenate((trainY_building,trainY_forest, trainY_glacier,trainY_mount, trainY_sea,trainY_street),axis=0)

## 1.2) test set

In [5]:
testX_building, testY_building  = convert("input/intel-image-classification/seg_test/seg_test/buildings/",0)
testX_forest,testY_forest  = convert("input/intel-image-classification/seg_test/seg_test/forest/",1)
testX_glacier,testY_glacier  = convert("input/intel-image-classification/seg_test/seg_test/glacier/",2)
testX_mount,testY_mount  = convert("input/intel-image-classification/seg_test/seg_test/mountain/",3)
testX_sea,testY_sea  = convert("input/intel-image-classification/seg_test/seg_test/sea/",4)
testX_street,testY_street  = convert("input/intel-image-classification/seg_test/seg_test/street/",5)

print('test building shape ', testX_building.shape, testY_building.shape) 
print('test forest', testX_forest.shape ,testY_forest.shape)
print('test glacier', testX_glacier.shape,testY_glacier.shape)
print('test mountain', testX_mount.shape, testY_mount.shape)
print('test sea',     testX_sea.shape, testY_sea.shape)
print('test street', testX_street.shape ,testY_street.shape)

test building shape  (437, 150, 150, 3) (437,)
test forest (474, 150, 150, 3) (474,)
test glacier (553, 150, 150, 3) (553,)
test mountain (525, 150, 150, 3) (525,)
test sea (510, 150, 150, 3) (510,)
test street (501, 150, 150, 3) (501,)


In [6]:
X_test= np.concatenate((testX_building,testX_forest, testX_glacier,testX_mount, testX_sea,testX_street),axis=0)
y_test= np.concatenate((testY_building,testY_forest, testY_glacier,testY_mount, testY_sea,testY_street),axis=0)

In [7]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((14034, 150, 150, 3), (3000, 150, 150, 3), (14034,), (3000,))

In [8]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_train.shape,y_test.shape
y_test[1]

array([1., 0., 0., 0., 0., 0.], dtype=float32)

# 2) modeling ResNet

In [9]:
from tensorflow.keras.layers import Input,Dropout, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from tensorflow.keras.models import Model, load_model,Sequential
from tensorflow.keras.initializers import glorot_uniform

In [10]:
def identity_block(X, f, filters, stage, block):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value. We'll need this later to add back to the main path. 
    X_shortcut = X

    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [11]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    
    ##### SHORTCUT PATH ####
    X_shortcut = Conv2D(F3, (1, 1), strides = (s,s), name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [12]:

def ResNet50(input_shape = (150, 150, 3), classes = 6):   
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL.
    X = AveragePooling2D((2, 2), name='avg_pool')(X)

    # output layer
    X = Flatten()(X)
    X = Dropout(0.3)(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [13]:
model = ResNet50(input_shape = (150, 150, 3), classes = 6)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 156, 156, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 75, 75, 64)   9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 75, 75, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

# 3) Training

In [14]:
model.fit(X_train, y_train, epochs = 50, batch_size = 64,validation_data=(X_test, y_test), shuffle=True)

Train on 14034 samples, validate on 3000 samples
Epoch 1/50
14034/14034 [==============================] - 61s 4ms/sample - loss: 1.4782 - acc: 0.5551 - val_loss: 3.4403 - val_acc: 0.1997
Epoch 2/50
14034/14034 [==============================] - 51s 4ms/sample - loss: 0.7030 - acc: 0.7448 - val_loss: 2.6877 - val_acc: 0.3373
Epoch 3/50
14034/14034 [==============================] - 51s 4ms/sample - loss: 0.6164 - acc: 0.7814 - val_loss: 1.1918 - val_acc: 0.6380
Epoch 4/50
14034/14034 [==============================] - 51s 4ms/sample - loss: 0.5304 - acc: 0.8103 - val_loss: 1.6693 - val_acc: 0.4993
Epoch 5/50
14034/14034 [==============================] - 51s 4ms/sample - loss: 0.4675 - acc: 0.8333 - val_loss: 1.2883 - val_acc: 0.6150
Epoch 6/50
14034/14034 [==============================] - 51s 4ms/sample - loss: 0.4481 - acc: 0.8402 - val_loss: 4.2950 - val_acc: 0.2563
Epoch 7/50
14034/14034 [==============================] - 51s 4ms/sample - loss: 0.4029 - acc: 0.8565 - val_loss: 0.6

In [15]:
model.save('/my_model.h5')
model.evaluate(X_test, y_test)

3000/3000 [==============================] - 4s 1ms/sample - loss: 0.9740 - acc: 0.8377


[0.9740369226181259, 0.8376667]